In [1]:
from holodecml.data import load_raw_datasets, load_unet_datasets
from holodecml.losses import unet_loss
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [2]:
path_data = "/glade/p/cisl/aiml/ai4ess_hackathon/holodec/"
num_particles = "medium"
output_cols = ["x", "y", "z", "d", "hid"]
subset = False
scaler_out = MinMaxScaler()
num_bins = False


In [7]:
np.sum(train_outputs[0, :, :, 0])

13.0

In [3]:
train_inputs_raw, train_outputs_raw = load_raw_datasets(path_data, num_particles,
                                                'train', output_cols, subset)


In [4]:
valid_inputs_raw, valid_outputs_raw = load_raw_datasets(path_data, num_particles,
                                                'valid', output_cols, subset)


In [ ]:
hid = 0
print(len(np.where(train_outputs_raw["hid"] == hid + 1)[0]))
fig, ax = plt.subplots(figsize=(18, 12))
ax.imshow(train_inputs_raw[hid].T, cmap='gray', vmin=0, vmax=255)
# ax.title.set_text(f'')
ax.set_xticks([])
ax.set_yticks([])
plt.show()


In [ ]:
def plot_hologram(h, inputs, outputs):
    """
    Given a hologram number, plot hologram and particle point
    
    Args: 
        h: (int) hologram index
        inputs: (pd df) input images
        outputs: (pd df) output x, y, z, and d values by hid
    
    Returns:
        print of pseudocolor plot of hologram and hologram particles
    """    
    x_vals = np.linspace(-888*2, 888*2, inputs[h, :, :].shape[0])
    y_vals = np.linspace(-592*2, 592*2, inputs[h, :, :].shape[1])

    plt.figure(figsize=(12, 8))
    plt.pcolormesh(x_vals, y_vals, inputs[h, :, :].T, cmap="RdBu_r")
    h_particles = np.where(outputs["hid"] == h +1)[0]
    for h_particle in h_particles:
        plt.scatter(outputs.loc[h_particle, "x"],
                    outputs.loc[h_particle, "y"],
                    outputs.loc[h_particle, "d"],
                    outputs.loc[h_particle, "z"],
                    vmin=outputs["z"].min(),
                    vmax=outputs["z"].max(),
                    cmap="cool")
        plt.annotate(f"d: {outputs.loc[h_particle,'d']:.1f} µm",
                     (outputs.loc[h_particle, "x"], outputs.loc[h_particle, "y"]))
    plt.xlabel("horizontal particle position (µm)", fontsize=16)
    plt.ylabel("vertical particle position (µm)", fontsize=16)
    plt.title(f"Hologram and particle positions plotted in four dimensions: {h_particles.shape[0]} particles", fontsize=20, pad=20)
    plt.colorbar().set_label(label="z-axis particle position (µm)", size=16)

In [ ]:
h = 1
plot_hologram(h, train_inputs_raw, train_outputs_raw)

In [ ]:
# toy inputs/outputs
xs = np.array([-10.5, 1.0, -9.5, -10.0, -9.5, 0.5, 1.5, -3.5, 6.5, 9.1, 10, 10.5])
ys = np.array([-3.5, 2.0, -0.5, -1.5, -1.0, -4.0, 2.5, 0.5, -2.0, 3.0, 5, 4.5])
zs = np.array([90, -100, 35, -65, -40, 0, -20, 80, 100, -11, 23, 88])
ds = np.array([1.1, 0.5, 2.0, 3.5, 0.4, 0.01, 3.9, 2.3, 1.16, 1.95, 0.75, 0.25])


In [ ]:
train_coords = []
train_outputs = []
for hid in train_outputs_raw["hid"].unique()[:2]:
    outputs_hid = train_outputs_raw.loc[train_outputs_raw['hid'] == hid]
    xs_hid = np.digitize(outputs_hid['x'], np.linspace(-888*2, 888*2, train_inputs_raw.shape[0]))
    ys_hid = np.digitize(outputs_hid['y'], np.linspace(-592*2, 592*2, train_inputs_raw.shape[1]))
    zs_hid = outputs_hid['z'].values
    ds_hid = outputs_hid['d'].values
    coords = np.array(list(zip(xs_hid,ys_hid)))
    coords = coords[np.lexsort((coords[:,1], coords[:,0]))]
    unique, unique_idx, unique_counts = np.unique(coords, return_index=True, return_counts=True, axis=0)
    for i in np.argwhere(unique_counts > 1):
        # find indices in original coordinates where there are multiple particles
        idx_equal = np.where((coords == unique[i][0]).all(axis=1))[0]
        # find the index of the particle with the z that is closest to the camera
        idx_max = np.argmin(zs_hid[np.min(idx_equal):np.max(idx_equal)+1])
        unique_idx[i] = idx_equal[idx_max]
    zs_hid = zs_hid[unique_idx]
    ds_hid = ds_hid[unique_idx]
    train_coords.append(coords)
    train_outputs.append(np.stack((zs_hid, ds_hid), axis=1))
# train_outputs_unet = np.stack(train_outputs_unet, axis=-1)
# train_outputs_unet.shape

In [ ]:
image = np.zeros((train_inputs_raw[0].shape[1], train_inputs_raw[0].shape[0]))
image[train_coords[1][:, 0], train_coords[1][:, 1]] = zs_hid
plt.imshow(image, interpolation='bilinear', cmap=plt.cm.gray, aspect='auto')
plt.show()

In [ ]:
train_coords[0][0]

In [ ]:
image[3:6, 842:847]

In [ ]:
# How sparse is this image... very 
fig=plt.figure(figsize=(12, 8))
image = np.zeros((train_inputs_raw[0].shape[1], train_inputs_raw[0].shape[0]))
image[train_coords[0][:, 0], train_coords[0][:, 1]] = 1
plt.imshow(image, interpolation='bilinear', cmap=plt.cm.gray, aspect='auto', vmin=0, vmax=1)
plt.show()

In [6]:
train_inputs, train_outputs, valid_inputs, valid_outputs = load_unet_datasets(path_data, num_particles, output_cols,
                                                                              scaler_out, subset, num_bins)

In [ ]:
def plot_binned_hologram(h, inputs, coords, outputs):
    """
    Given a hologram number, plot hologram and particle point
    
    Args: 
        h: (int) hologram index
        inputs: (pd df) input images
        outputs: (pd df) output x, y, z, and d values by hid
    
    Returns:
        print of pseudocolor plot of hologram and hologram particles
    """    
    x_vals = np.linspace(-888*2, 888*2, inputs[h, :, :].shape[0])
    y_vals = np.linspace(-592*2, 592*2, inputs[h, :, :].shape[1])

    plt.figure(figsize=(12, 8))
    plt.pcolormesh(x_vals, y_vals, inputs[h, :, :].T, cmap="RdBu_r")
    for h_particle in range(outputs[h].shape[0]):
        plt.scatter(np.linspace(-888*2, 888*2, train_inputs_raw.shape[0])[coords[h][h_particle][0]],
                    np.linspace(-592*2, 592*2, train_inputs_raw.shape[1])[coords[h][h_particle][1]],
                    outputs[h][h_particle][1],
                    outputs[h][h_particle][0],
                    vmin=outputs[h][:,1].min(),
                    vmax=outputs[h][:,1].max(),
                    cmap="cool")
        plt.annotate(f"d: {outputs[h][h_particle][1]:.1f} µm",
                     (np.linspace(-888*2, 888*2, train_inputs_raw.shape[0])[coords[h][h_particle][0]],
                      np.linspace(-592*2, 592*2, train_inputs_raw.shape[1])[coords[h][h_particle][1]]))
    plt.xlabel("horizontal particle position (µm)", fontsize=16)
    plt.ylabel("vertical particle position (µm)", fontsize=16)
    plt.title(f"Hologram and particle positions plotted in four dimensions: {outputs[h].shape[0]} particles", fontsize=20, pad=20)
    plt.colorbar().set_label(label="z-axis particle position (µm)", size=16)

In [ ]:
h = 1
plot_binned_hologram(h, train_inputs_raw, train_coords, train_dz)

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    BatchNormalization,
    Conv2D,
    Conv2DTranspose,
    MaxPooling2D,
    Dropout,
    SpatialDropout2D,
    UpSampling2D,
    Input,
    concatenate,
    multiply,
    add,
    Activation,
)
from tensorflow.keras.optimizers import Adam, SGD


In [5]:
def upsample_conv(filters, kernel_size, strides, padding):
    return Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)


def upsample_simple(filters, kernel_size, strides, padding):
    return UpSampling2D(strides)


def attention_gate(inp_1, inp_2, n_intermediate_filters):
    """Attention gate. Compresses both inputs to n_intermediate_filters filters before processing.
       Implemented as proposed by Oktay et al. in their Attention U-net, see: https://arxiv.org/abs/1804.03999.
    """
    inp_1_conv = Conv2D(
        n_intermediate_filters,
        kernel_size=1,
        strides=1,
        padding="same",
        kernel_initializer="he_normal",
    )(inp_1)
    inp_2_conv = Conv2D(
        n_intermediate_filters,
        kernel_size=1,
        strides=1,
        padding="same",
        kernel_initializer="he_normal",
    )(inp_2)

    f = Activation("relu")(add([inp_1_conv, inp_2_conv]))
    g = Conv2D(
        filters=1,
        kernel_size=1,
        strides=1,
        padding="same",
        kernel_initializer="he_normal",
    )(f)
    h = Activation("sigmoid")(g)
    return multiply([inp_1, h])


def attention_concat(conv_below, skip_connection):
    """Performs concatenation of upsampled conv_below with attention gated version of skip-connection
    """
    below_filters = conv_below.get_shape().as_list()[-1]
    attention_across = attention_gate(skip_connection, conv_below, below_filters)
    return concatenate([conv_below, attention_across])


def conv2d_block(
    inputs,
    use_batch_norm=True,
    dropout=0.3,
    dropout_type="spatial",
    filters=16,
    kernel_size=(3, 3),
    activation="relu",
    kernel_initializer="he_normal",
    padding="same",
):

    if dropout_type == "spatial":
        DO = SpatialDropout2D
    elif dropout_type == "standard":
        DO = Dropout
    else:
        raise ValueError(
            f"dropout_type must be one of ['spatial', 'standard'], got {dropout_type}"
        )

    c = Conv2D(
        filters,
        kernel_size,
        activation=activation,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=not use_batch_norm,
    )(inputs)
    if use_batch_norm:
        c = BatchNormalization()(c)
    if dropout > 0.0:
        c = DO(dropout)(c)
    c = Conv2D(
        filters,
        kernel_size,
        activation=activation,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=not use_batch_norm,
    )(c)
    if use_batch_norm:
        c = BatchNormalization()(c)
    return c


In [6]:
def custom_unet(
    input_shape,
    num_classes=1,
    activation="relu",
    use_batch_norm=True,
    upsample_mode="deconv",  # 'deconv' or 'simple'
    dropout=0.3,
    dropout_change_per_layer=0.0,
    dropout_type="spatial",
    use_dropout_on_upsampling=False,
    use_attention=False,
    filters=16,
    num_layers=3,
    output_activation="sigmoid",
):  # 'sigmoid' or 'softmax'

    """
    Customisable UNet architecture (Ronneberger et al. 2015 [1]).
    Arguments:
    input_shape: 3D Tensor of shape (x, y, num_channels)
    num_classes (int): Unique classes in the output mask. Should be set to 1 for binary segmentation
    activation (str): A keras.activations.Activation to use. ReLu by default.
    use_batch_norm (bool): Whether to use Batch Normalisation across the channel axis between convolutional layers
    upsample_mode (one of "deconv" or "simple"): Whether to use transposed convolutions or simple upsampling in the decoder part
    dropout (float between 0. and 1.): Amount of dropout after the initial convolutional block. Set to 0. to turn Dropout off
    dropout_change_per_layer (float between 0. and 1.): Factor to add to the Dropout after each convolutional block
    dropout_type (one of "spatial" or "standard"): Type of Dropout to apply. Spatial is recommended for CNNs [2]
    use_dropout_on_upsampling (bool): Whether to use dropout in the decoder part of the network
    use_attention (bool): Whether to use an attention dynamic when concatenating with the skip-connection, implemented as proposed by Oktay et al. [3]
    filters (int): Convolutional filters in the initial convolutional block. Will be doubled every block
    num_layers (int): Number of total layers in the encoder not including the bottleneck layer
    output_activation (str): A keras.activations.Activation to use. Sigmoid by default for binary segmentation
    Returns:
    model (keras.models.Model): The built U-Net
    Raises:
    ValueError: If dropout_type is not one of "spatial" or "standard"
    """

    if upsample_mode == "deconv":
        upsample = upsample_conv
    else:
        upsample = upsample_simple

    # Build U-Net model
    inputs = Input(input_shape)
    x = inputs

    down_layers = []
    for l in range(num_layers):
        x = conv2d_block(
            inputs=x,
            filters=filters,
            use_batch_norm=use_batch_norm,
            dropout=dropout,
            dropout_type=dropout_type,
            activation=activation,
        )
        down_layers.append(x)
        x = MaxPooling2D((2, 2))(x)
        dropout += dropout_change_per_layer
        filters = filters * 2  # double the number of filters with each layer

    x = conv2d_block(
        inputs=x,
        filters=filters,
        use_batch_norm=use_batch_norm,
        dropout=dropout,
        dropout_type=dropout_type,
        activation=activation,
    )

    if not use_dropout_on_upsampling:
        dropout = 0.0
        dropout_change_per_layer = 0.0

    for conv in reversed(down_layers):
        filters //= 2  # decreasing number of filters with each layer
        dropout -= dropout_change_per_layer
        x = upsample(filters, (2, 2), strides=(2, 2), padding="same")(x)
        if use_attention:
            x = attention_concat(conv_below=x, skip_connection=conv)
        else:
            x = concatenate([x, conv])
        x = conv2d_block(
            inputs=x,
            filters=filters,
            use_batch_norm=use_batch_norm,
            dropout=dropout,
            dropout_type=dropout_type,
            activation=activation,
        )

    outputs = Conv2D(num_classes, (1, 1), activation=output_activation)(x)

    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model


In [7]:
input_shape = np.expand_dims(train_inputs, axis=-1).shape[1:]
model = custom_unet(
    input_shape,
    use_batch_norm=True,
    num_classes=3,
    filters=64,
    dropout=0.2,
    output_activation='sigmoid',
    use_attention=True
)


In [8]:
model.compile(
    optimizer=Adam(lr=0.0001),
    loss=unet_loss
)


In [9]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 400, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 600, 400, 64) 576         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 600, 400, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
spatial_dropout2d (SpatialDropo (None, 600, 400, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [10]:
history = model.fit(
    np.expand_dims(train_inputs, axis=-1),
    train_outputs,
    batch_size=8,
    epochs=10,
    validation_data=(np.expand_dims(valid_inputs, axis=-1), valid_outputs)
)


Epoch 1/10


OperatorNotAllowedInGraphError: in user code:

    /glade/work/ggantos/ncar_20200417/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /glade/work/ggantos/holodec-ml/holodecml/losses.py:100 unet_loss  *
        print("MAX", max(y_pred))
    /glade/work/ggantos/ncar_20200417/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:503 __iter__  **
        self._disallow_iteration()
    /glade/work/ggantos/ncar_20200417/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:496 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    /glade/work/ggantos/ncar_20200417/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:474 _disallow_when_autograph_enabled
        " indicate you are trying to use an unsupported feature.".format(task))

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did conve

In [ ]:
valid_outputs_pred = model.predict(np.expand_dims(valid_inputs, axis=-1),
                                     batch_size=32)


In [ ]:
import xarray as xr

valid_outputs_pred_da = xr.DataArray(valid_outputs_pred,
                                     coords={"hid": np.arange(valid_outputs_pred.shape[0]),
                                             "x": np.arange(valid_outputs_pred.shape[1]),
                                             "y": np.arange(valid_outputs_pred.shape[2]),
                                             "output": ["p", "z", "d"]},
                                     dims=("hid", "x", "y", "output"),
                                     name="valid_pred_scaled")

In [ ]:
from os.path import join

path_save = "/glade/p/cisl/aiml/ggantos/holodec/unet/test_jupyter_fixed/"
model.save(path_save, save_format="tf")
model.save_weights(path_save + '_weights', save_format='tf')
valid_outputs_pred_da.to_netcdf(join(path_save, "valid_outputs_pred.nc"))
for k in history.history.keys():
    np.savetxt(join(path_save, k+".csv"), history.history[k])


In [ ]:
valid_outputs_pred_da

In [ ]:
valid_outputs_pred.shape

In [ ]:
valid_outputs_pred[0, :, :, 0].shape

In [ ]:
valid_outputs[0, :, :, 0].shape

In [ ]:
fig=plt.figure(figsize=(12, 8))
plt.imshow(valid_outputs[0, :, :, 0], interpolation='bilinear', cmap=plt.cm.gray, aspect='auto', vmin=0, vmax=1)
print(np.sum(valid_outputs[0, :, :, 0]))
plt.show()

In [ ]:
fig=plt.figure(figsize=(12, 8))
plt.imshow(valid_outputs_pred[0, :, :, 0], interpolation='bilinear', cmap=plt.cm.gray, aspect='auto', vmin=0, vmax=1)
print(np.sum(valid_outputs_pred[0, :, :, 0]))
plt.show()

In [ ]:
from holodecml.losses import unet_loss

In [ ]:
unet_loss(valid_outputs[0, :, :, 0], valid_outputs_pred[0, :, :, 0])

In [ ]:
import tensorflow as tf
bce = valid_outputs[0, :, :, 0] * -tf.math.log(valid_outputs_pred[0, :, :, 0]+1e-8)

In [ ]:
bce

In [ ]:
tf.reduce_sum(bce)

In [ ]:
bce_2 = (1 - valid_outputs[0, :, :, 0]) * -tf.math.log(1 - valid_outputs_pred[0, :, :, 0]+1e-8)
tf.reduce_sum(bce_2)

In [ ]:
(1 - valid_outputs[0, :, :, 0]).shape

In [ ]:
np.sum((1 - valid_outputs[0, :, :, 0]))

In [ ]:
import tensorflow as tf
-tf.math.log(np.max(1 - valid_outputs_pred[0, :, :, 0], 1e-8))

In [ ]:
valid_outputs_pred[0, :, :, 0]

In [ ]:
x = np.array([[0, 1.3, 2],
             [1, 5.4, 0]])

In [ ]:
x

In [ ]:
np.max(x, 1e-8)

In [ ]:
x[x < 1e-8] = 1e-8

In [ ]:
x